### Fitting the Zero model

This is a notebook with the zero model fitting. The process is analogous with the previous notebook, except we now define a zero model instad of the ALP model. 

In [1]:
# importing general packages

import numpy as np
import os, sys
import matplotlib.pyplot as plt

import xspec as xs
xs.Xset.allowPrompting = False #keeps pyxspec from hanging, waiting a response to a prompt
xs.Xset.allowNewAttributes = True

In [2]:
SED10=np.load('bn121225417_SED10.npy')

def zero(engs, params, flux):
    norm1 = params[0]
    flux[:] = norm1*np.ones(len(SED10))*1e-150  # an array of small numbers ~0
    
zeroInfo = ("norm1 \"\" 1 1e-1  1e-1  1e1 1e1 1",)
xs.AllModels.addPyMod(zero, zeroInfo, 'add',  spectrumDependent=True)

In [3]:
# normalizations, coarse grid
N = np.logspace(np.log10(8.4e-8), np.log10(8.4e2), 30)

normZERO = np.zeros((30,2001))
chi_stat = np.zeros((30,2001))
Fit_st = np.zeros((30,2001))

xs.AllData.clear()   # clear all data, if any.

for j in range(30):
    for i in range(1,2001):
        xs.AllData.clear()
        s=xs.Spectrum("bn121225417_ALP%s_bkg.fak{%s}" %(j,i))
        s.response = "bn121225417_LAT-LLE_weightedrsp.rsp"
        s.background= ("bn121225417_LAT-LLE_bkgspectra.bak{1}")
        xs.AllModels.clear()
        m=xs.Model("zero")
        xs.Fit.statMethod = "pgstat"
        m.zero.norm1.values=1.
        m.zero.norm1.frozen = True
        m.zero.norm.frozen = False
        xs.Fit.perform()
        chi_stat[j,i] = xs.Fit.testStatistic
        Fit_st[j,i] = xs.Fit.statistic  
        
np.save('TS_pgfit_ZERO.npy', chi_stat)
np.save('pgfit_ZERO.npy', Fit_st)     